<a href="https://colab.research.google.com/github/doeungim/ADP_DE/blob/main/Non_Parametric_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 피셔의 정확검정

- 샘플의 크기가 적을 때, "범주형" 변수간의 독립성 평가
- 카이제곱의 비모수

In [78]:
from scipy.stats import fisher_exact

data = [[10,2],[3,6]]
## 샘플의 크기가 작음

odds, p_val = fisher_exact(data)
print(p_val)

0.03184431667403804


# RUN 검정 : 데이터의 순서의 무작위성 존재를 검
### 1. 일표본 RUN 검정 : 한 개의 샘플이 무작위로 추출되었는지에 대한 여부 (RUN)이란 동일한 관측값이 연속적으로 이어진 것
- 범주형 : 범주의 개수와  RUN 의 개수
- 수치형 : 중앙값을 기준으로 데이터를 이진화 후 검정

[가설 검정] </br>
H0 : 샘플이 무작위로 추출되었다. </br>
H1 : 샘플이 무작위로 추출되지 않았다.


In [1]:
## 범주형 데이터의 경우
from collections import Counter
import numpy as np

data = ['a', 'a', 'b', 'b', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'a', 'b', 'a', 'a', 'b', 'b', 'a', 'a', 'b', 'b', 'b', 'a', 'a', 'b', 'b', 'a', 'b']
data2 = list(map(lambda x : 1 if x == 'a' else 0 , data))

## 범주형
from statsmodels.sandbox.stats.runs import runstest_1samp

z_stat, p_val = runstest_1samp(data2)
print(f"{p_val} 의 유의 수준으로 귀무가설 채택")


0.7097857460625617 의 유의 수준으로 귀무가설 채택


In [2]:
## 수치형
data = [50, 60, 70, 40, 30, 20, 10, 70, 80, 100]
median = np.median(data)

data2 = []
for sample in data :
  if sample >= median :
    data2.append(1)
  else :
    data2.append(0)

z_stat, p_val2 = runstest_1samp(data2, cutoff = 'median')
print(round(p_val2,4))

0.3143


### 2. 이표본 RUN 검정

[가설 검정] </br>
H0 : 두 데이터는 같은 분포이다. </br>
H1 : 두 데이터는 같은 분포가 아니다.

In [3]:
# 수치형 - 1,0으로 치환
data1 = [23, 42, 36, 27, 48, 52, 35, 31]
data2 = [43, 56, 38, 20, 46, 51, 36]


## 1, 0으로 치환 : float(x) - 데이터 타입을 변경해준다.
data1 = list(map(lambda x: float(x), data1))
data2 = list(map(lambda x: float(x), data2))

# data1 + data2
data3 = data1 + data2
med = np.median(data3)
data3 = [1 if i >= med else 0 for i in data3]

from statsmodels.sandbox.stats.runs import runstest_2samp

z_stat, p_val = runstest_2samp(data1, data2)
print(p_val)

ties detected
0.9856795756756671


# 이항 변수 검정
### 1. 맥니머 : 대응 관계가 있는 2개의 "범주형의 이항"변수의 "분포"검정, 사전 및 사후 관찰에서 두 "범주"간 변화가 유의미한지 검증한다.

[가설 검정] </br>
H0 : 두 변수의 데이터 분포는 같다. </br>
H1 : 두 변수의 데이터 분포는 같지 않다.

### 2. 코크란 Q : 대응 관계가 있는 K개의 "이항"변수의 "분포"검정, 여러 처치나 시간 대에 따른 이진 데이터의 변화가 유의미한지 검정
데이터가 1,0 (있다, 없다)로 이루어짐 </br>
[가설 검정] </br>
H0 : 두 변수의 데이터 분포는 같다. </br>
H1 : 두 변수의 데이터 분포는 같지 않다.

In [4]:
# 맥니머

from pandas import DataFrame
table = DataFrame([[9, 12], [24, 35]], index=['전_있음', '전_없음'], columns=['후_있음', '후_없음'])
print(f"[데이터 확인]\n{table}")

# 예제 1)
# H0 : 흥미가 없다가 있음 VS 흥미가 있다가 없음의 분포가 같다.
from statsmodels.stats.contingency_tables import mcnemar

mc = mcnemar(table.values, exact = False, correction = False)
print(f"{mc.pvalue}","귀무가설 기각" if mc.pvalue < 0.05 else "귀무가설 채택")

# 예제 2)
from statsmodels.stats.contingency_tables import mcnemar
# 예제 데이터: 사전 및 사후 이진 결과
table = [[10, 5], [3, 12]]  # (사전: [yes, no], 사후: [yes, no])

result = mcnemar(table, exact=False)
display(result.pvalue)


[데이터 확인]
      후_있음  후_없음
전_있음     9    12
전_없음    24    35
0.04550026389635857 귀무가설 기각


0.7236736098317629

In [5]:
## 2. 코크란 q
from pandas import DataFrame
import numpy as np
from scipy.stats import chi2
table = DataFrame([[0,1,0,1,0,0,0,0], [1,1,0,1,0,0,1,1], [0,1,1,1,1,1,1,1]], index=['가수1', '가수2', '가수3'], columns=[1,2,3,4,5,6,7,8]).T
display(table.head(3))

# 예제1)
# H0 : 모든 연예인들에 대한 호감도는 같다.
# H1 : 적어도 하나의 연예인에 대한 호감도는 다르다.
from statsmodels.stats.contingency_tables import cochrans_q
ccq = cochrans_q(table)
print(" 검정통계량 {:.3f}, p-value {:.3f}".format(ccq.statistic, ccq.pvalue))

,가수1,가수2,가수3
1,0,1,0
2,1,1,1
3,0,0,1


 검정통계량 6.333, p-value 0.042


# 부호, 순위 검정
#### 일표본, 이표본 부호 검정 (대응 표본 t 검정의 비모수적 대안)
### 1. 일표본 부호 검정 : 이항 분포, Z 분포
- n < 100 이항 분포이며 N > 100이면, Z 분포
- 모집단의 중위수가 특정 값과 다른지 검정
- ex) 10명 환자의 치료 전후 차이 점수가 0보다 큰지 검정

In [6]:
import numpy as np
data = np.array([203,204,197,195,201,205,198,199,194,207]) #데이터
M0 = 200  # 중위수


# H0 : 데이터의 중앙값은 200과 차이가 없다.
# H1 : 데이터의 중앙값은 200과 차이가 있다.

# Perform sign test
from scipy.stats import binomtest
# data 와 M0의 차이에 SING으로 양수면 1, 음수면 -1
signs = np.sign(data - M0)
positive_signs = np.sum(signs > 0)
n = np.sum(signs != 0)

# Binomial test
p_value = binomtest(positive_signs, n=n, p=0.5, alternative='two-sided')
print(f'One-Sample Sign Test p-value: {p_value}')
signs

One-Sample Sign Test p-value: BinomTestResult(k=5, n=10, alternative='two-sided', statistic=0.5, pvalue=1.0)


array([ 1,  1, -1, -1,  1,  1, -1, -1, -1,  1])

In [7]:
# 예제2) 문제
#아래 데이터의 중앙값이 130이라고 할 수 있는지 검정하라
d = np.array([133,145,155,162,143,133,166])
m = 130

sign = np.sign(d-m)
positive_sign = np.sum(sign > 0)
n = np.sum(sign != 0)

p_val = binomtest(positive_sign, n = n, p = 0.5, alternative = 'two-sided')
print(p_val)

BinomTestResult(k=7, n=7, alternative='two-sided', statistic=1.0, pvalue=0.015625)


### 2. 이표본 부호 검정 : 이항 분포, Z 분포
- n < 100 이항 분포이며 N > 100이면, Z 분포
- 두 관련된 표본의 "중앙값의 차이"가 유의미한지 검정
- ex) 두 치료 방법의 효과를 비교

In [8]:
# 예제1) 두 표본의 중위수가 특정값과 다른지 검정
# H0 : A,B의 만족도 차이는 없다.
# H1 : A,B의 만족도 차이는 있다.

data1 = [4,3,5,2,1,3,4,3] #A
data2 = [3,2,3,1,2,2,2,2]

signs = [1 if d1 > d2 else 0 for d1, d2 in zip(data1, data2)]
p_val = binomtest(sum(signs), n = len(signs), p = 0.5 , alternative = 'two-sided')
print(p_val)



# 예제 2)
data3 =  [5, 10, 15, 20]
data4 = [4, 12, 14, 21]

s = [1 if d3 > d4 else 0 for d3, d4 in zip(data3, data4)]
p_val2 = binomtest(sum(s) , n = len(s), p = 0.5, alternative = 'two-sided')
print(p_val2)

BinomTestResult(k=7, n=8, alternative='two-sided', statistic=0.875, pvalue=0.0703125)
BinomTestResult(k=2, n=4, alternative='two-sided', statistic=0.5, pvalue=1.0)


# 부호, 순위 검정
#### 일표본, 이표본 부호/순위 검정 : Paired의 중앙값 차이 검정 (순위데이터)
### 1. 일표본 wilcoxon 부호순위 검정 :  
- n < 20 "wilcoxon" 이며 N > 20 이면, Z 분포
- 대응 관계의 "양적"데이터의 두 그룹의 차이를 검정


In [9]:
from scipy.stats import wilcoxon

# H0 : 데이터 중위수 = 200
# H1 : 데이터 중위수 != 200
data = np.array([203,204,197,195,201,205,198,199,194,207]) #데이터
M0 = 200 #중앙값

stat, p_value = wilcoxon(M0 -data)
print(f'Wilcoxon statistic: {stat}, p-value: {p_value}')

stats, p = wilcoxon([M0 for i in range(len(data))], data)
print(stats, p)

Wilcoxon statistic: 25.5, p-value: 0.845703125
25.5 0.845703125


In [10]:
import pandas as pd
import scipy.stats as stats
# 이표본
# 예제2) 다이어트 약의 효과를 비교하기 위해 투약 전후의 중성지방 투약 전후 변화가 존재 하는지 검정하라

# 접근 방법 :
# 1) 전 후의 대응 표본
# 2) N수가 20개 미만 , 숫자형 데이터

# H0 : 전후 차이가 없다.
# H1 : 전후 차이가 있다.

data = pd.DataFrame({'before': [250, 180, 160, 145, 130, 120] ,
                     'after': [120, 155, 145, 125, 135, 130]})
data.index = ['Trial_' + str(i)  for i in range(1,7)]
display(data)

before = [1250, 180, 160, 145, 130, 120]
after = [120, 155, 145, 125, 135, 130]

result = stats.wilcoxon(after, before)
print(result.pvalue)

,before,after
Trial_1,250,120
Trial_2,180,155
Trial_3,160,145
Trial_4,145,125
Trial_5,130,135
Trial_6,120,130


0.15625


In [11]:
# 이표본
# 예제2) 1번째, 2번째 측정값의 차이가 있는지 없는지 검정
data1 = np.array([79, 96, 85, 69, 88, 75, 83, 88])
data2 = np.array([70, 88, 73, 74, 75, 79, 77, 81])

stat, p = wilcoxon(data1, data2, zero_method = 'wilcox')
print(p)

0.0390625


#  부호 검정 VS 윌콕슨 부호 순위 검정

1. 개념
  - 부호 : 쌍의 데이터의 차이, 그 차이가 양수, 음수인지
  - 윌콕슨 : 쌍의 데이터 차이 그 차이의 절대값에 순위를 부여, 순위에 부효를 붙여 양수, 음수순의 합을 비교
  - **단순히 처리 전후에 변화**가 있는지 여부만 알고 싶다면 **일표본 부호검정**
  - **차이의 크기**에 대한 정보가 중요하다면 **윌콕슨 부호 순위 검정**을 사용

# 맨휘트니-u검정-윌콕슨-순위합-검정
- n(=n1 + n2) < 25 이면, 순위합 분포  (use_continuity=True)
- n(=n1 + n2) > 25 이면, 독립표본 t검정에 대응  (use_continuity=False)
- 대응 관계의 "양적/순위"데이터의 "두 그룹의 차이"를 검정
- 두 그룹의 중앙값이 같은 지 비교
- 분포가 정규 분포를 따르지 않을 때
```
from scipy.stats import mannwhitneyu
mannwhitneyu(
    x,
    y,
    use_continuity=True,
    alternative='two-sided',
    axis=0,
    method='auto',
)
```

In [12]:
# 예제1) 팀 영업별 성적의 차이가 있는지 검정
# H0 : A,B 중앙값의 차이는 0 이다.
# H1 : A,B 중앙값의 차이는 0 이 아니.

data1 = [87,75,65,95,90,81,93]
data2 = [57,85,90,83,87,71]

print("[라이브러리 검정]")
from scipy.stats import ranksums, mannwhitneyu
zstats, p = ranksums(data1, data2) #default: alternative='two-sided'
print(" 순위합 정규근사 검정통계량: {:.3f}, p-value: {:.3f}".format(zstats, p))


mstat, p = mannwhitneyu(data1, data2, alternative='two-sided')
print(" 만위트니 검정통계량: {:.3f}, p-value: {:.3f}".format(mstat, p))

[라이브러리 검정]
 순위합 정규근사 검정통계량: 0.857, p-value: 0.391
 만위트니 검정통계량: 27.000, p-value: 0.431


# 크러스컬-윌리스-검정
- K개 이상의 ANOVA 의 비모수 대한 (정규분포 X, 등분산성 X)
- 대응이 없는 여러 집단의 순서데이터
- 3 집단 : 17개 이하 / 4 집단 : 14 이하인 경우, 카이제곱 검정의 경우 : 검출표 기준으로 산정
- 단측 검정만 존재
from scipy.stats import kruskal
```

In [13]:
table = DataFrame([[69,67,65,59,66],
                    [56,63,55,40],
                    [71,72,70,75]],
                  index=['A','B','C'])


display(table)

# 귀무 : 세 사람의 모의고사의 점수 차이가 없다.

from scipy.stats import kruskal
print("[라이브러리 검정]")
# nan_policy='omit'으로 설정하면 nan값은 제거하고 자동 계산
stat, p = kruskal(table.values[0], table.values[1], table.values[2], nan_policy='omit')
print(" 검정통계량 {:.3f}, p-value {:.3f}".format(stat, p))

,0,1,2,3,4
A,69,67,65,59,66.0
B,56,63,55,40,NaN
C,71,72,70,75,NaN


[라이브러리 검정]
 검정통계량 10.118, p-value 0.006


# 프리드먼 검정
- 대응이 있는 여러 집단의 "순서"데이터 검정
- 대응이 있는 여러 집단의 "차이"- 일원배치 분산 분석
- 3개의 범주 :  N < 9
- 4개의 범수 : N < 5
- 데이터가 크면 카이제곱분포에 근사한다.

In [14]:
import numpy as np
from scipy.stats import kruskal

# 학습법 1, 2에 대한 시험 점수
study_method1 = np.array([90, 85, 88, 75, 80, 85, 80, 83, 70, 78])
study_method2 = np.array([88, 84, 86, 76, 82])

# 크러스컬-윌리스 검정 수행
stat, p_value = kruskal(study_method1, study_method2)

print(f"검정 통계량: {stat}")
print(f"p-값: {p_value}")

# p-값이 0.05보다 작으면 귀무가설 기각
if p_value < 0.05:
    print("귀무가설을 기각합니다. 학습법에 따른 점수 변화가 유의미합니다.")
else:
    print("귀무가설을 기각할 수 없습니다. 학습법에 따른 점수 변화는 유의미하지 않습니다.")


검정 통계량: 0.3053859964093437
p-값: 0.5805254212140851
귀무가설을 기각할 수 없습니다. 학습법에 따른 점수 변화는 유의미하지 않습니다.


In [15]:
from scipy.stats import friedmanchisquare
table = DataFrame([[4,2,5], [3,5,2], [5,4,4], [1,1,3]], index=['A', 'B', 'C', 'D'])
print(f"[데이터 확인]\n{table}")

# 귀무 : A,B,C,D의 운전 점수에 차이가 없다

stat, p = friedmanchisquare(table.values[0], table.values[1], table.values[2], table.values[3])
print(" 검정통계량 {:.3f}, p-value {:.3f}".format(stat, p))

[데이터 확인]
   0  1  2
A  4  2  5
B  3  5  2
C  5  4  4
D  1  1  3
 검정통계량 4.200, p-value 0.241


연습 문제 1) A,B,C 동별 주민 대표 찬반 투표 결과, 통별 찬반 비율이 동일한지 설정하고, 검정 통계량을 계산하여검정하시오

In [16]:
t= pd.DataFrame( {"구분" :['찬성', '반대'] ,
             "A" : [50,45],
             "B" :[60,32],
             "C" :[65,55],
             }).set_index('구분')
display(t)

from scipy.stats import chi2_contingency
chi, p_val , df, exp = chi2_contingency(t, correction = False)

print(p_val)


,A,B,C
구분,,,
찬성,50,60,65
반대,45,32,55


0.1598230083055444


연습 문제 2) 특정 캠패인에 노출된 전, 후의 데이터, 지지 비율의 차이가 있는지 가설 검정

In [17]:
from pandas import read_csv
data = read_csv('https://raw.githubusercontent.com/algoboni/pythoncodebook1-1/main/practice6_ba.csv', index_col=0)
print(data.head(5).T)

cross = pd.crosstab(data['before'], data['after'])

from statsmodels.stats.contingency_tables import mcnemar
mc = mcnemar(cross, exact = False, correction = False)
print()
print(mc.pvalue)

        0  1  2  3  4
before  1  1  1  0  1
after   0  1  1  1  0

0.5126907602619235


연습 문제 3) T. F가 무작위로 나열되어있다고 볼 수 있는지 검정

In [18]:
TFlist = ['T','F','F','T','F','T','F','T','T','F','F','T','F','T','F','T','F','T']

# 귀무 : T.F는 무작위로 나열되었다.
# 대립 : T.F는 무작위로 나열되지 않았다.

from statsmodels.sandbox.stats.runs import runstest_1samp

# 0,1 치환
data = list(map(lambda x : 1 if x == 'T' else 0, TFlist))

z_stat, p = runstest_1samp(data)
print("귀무가설 기각" if p < 0.05 else "귀무가설 채택")

귀무가설 기각


연습 문제 4) 20대 보다 30대가 기대하는 연봉 상승률이 높다는 주장이 있다. 20대 10명과 30대 8명을 임의로 추출하여 기대하는 연봉 상승률을 정리하였다.

In [19]:
# 귀무 : 20대와 30대의 기대하는 연봉 상승률은 같다.
# 대립 : 20대보다 30대가 기대한는 연봉 상승률이 더 낮다.

A = [3.0, 3.5, 2.0, 2.8, 5.0, 0, 2.3, 2.8, 3.3, 3.5] #20대
B = [3.5, 5.5, 5.0, 5.0, 10.0, 8.0, 2.5, 3.0 ]       #30대

mstat, p = mannwhitneyu(A, B, alternative='less')
print(" 만위트니 검정통계량: {:.3f}, p-value: {:.3f}".format(mstat, p))

 만위트니 검정통계량: 14.500, p-value: 0.013


연습 문제 5) 양식, 한식, 중식에 대한 5명의 선호도를 나타내는 데이터이다. 음식 종류에 따라 선호도의 차이가 있는지 검정

In [20]:
# 문제에서 제시된 표
data = pd.DataFrame(
    {'구분' : ['양식', '중식','한식'],
     'A' : [1,1,2] ,
     'B' : [3,3,3] ,
     "C" : [1,3,3] ,
     "D" : [3,1,3] ,
     "E" : [1,1,1]
     }


).set_index('구분')
display(data)

# 귀무 : 음식 종류에 대한 선호도 차이는 없다.
# 대립 : 적어도 하나의 음식에 대한 선호도 차이는 있다.
W = [1,3,1,3,1]
K = [1,3,3,1,1]
C = [2,3,3,3,1]

from scipy.stats import friedmanchisquare
fstat, p = friedmanchisquare(W, K, C)
print(p)

,A,B,C,D,E
구분,,,,,
양식,1,3,1,3,1
중식,1,3,3,1,1
한식,2,3,3,3,1


0.2635971381157259


연습 문제 6) 임의로 추출한 A,B,C 학생들의 키가 다음과 같다. 지역별 학생들의 키가 차이가 있는지 검정

In [21]:
## 크루스컬 월리스 검정
# 귀무가설: 세 지역 학생들의 키는 차이가 없다.
# 대립가설: 적어도 한쌍의 지역 학생들의 키는 차이가 있다.

A = [177, 167, 188, 189, 152, 159, 184, 175]
B = [151, 177, 150, 187, 167, 166, 179, 161, 174]
C = [173, 151, 156, 182, 188, 175, 150, 165, 176, 183]

from scipy.stats import kruskal
kstat, p = kruskal(A, B, C)
print("검정통계량 {:.3f}, p-value {:.3f}".format(kstat, p))

검정통계량 1.331, p-value 0.514


기출 예제 - ADP25)  공장 X,Y,Z의 평균 출하 소요시간을 여러 일자에 거쳐 측정한 데이터이다. 각 공장의 중위수의 차이가 존재하는지 확인하려 한다

In [22]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/25/problem5.csv',encoding='cp949')
display(df.head(3))
display(len(df))
# 공장명 집단 확인
df['공장명'].value_counts()

# 대응 관계 X
# 집단  3개 이상
# 적은 N개의 수 -- 크루스칼 (비모수)

X = df[df['공장명'] == 'X']['출하 소요시간']
Y = df[df['공장명'] == 'Y']['출하 소요시간']
Z = df[df['공장명'] == 'Z']['출하 소요시간']

from scipy.stats import kruskal

stat, p_val = stats.kruskal(X, Y, Z)
print(p_val)

,출하 소요시간,공장명,공장별 출하 소요시간 순위
0,71,Z,11
1,75,X,14
2,71,X,12


44

0.6970526702392181


기출 예제 ADP_26)

In [33]:
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/26/pressure.csv')
display(df.head(3))


# HO : 성별에 따른 혈압의 차이는 없다
# H1 : 성별데 따른 혈압의 차이는 있다.

m = df[df.gender == 'male']['pressure']
f = df[df.gender == 'female']['pressure']

# 정규성 검정
# 귀무가설 : 데이터는 정규 분포이다.

print(" 남자인 경우, 귀무가설 기각" if stats.shapiro(m).pvalue < 0.05 else "귀무가설 정규분포이다" )
print(" 여자인 경우, 귀무가설 기각" if stats.shapiro(f).pvalue < 0.05 else "귀무가설 정규분포이다" )

# 한쪽이 정규 분포가 아니다 > 비모수 진행
# 대응 관계가 없는 남/여 2 범주를 비교 - 만 위트니

from scipy.stats import mannwhitneyu

print(mannwhitneyu(m, f))


,gender,pressure
0,male,106.8
1,male,100.8
2,male,84.5


 남자인 경우, 귀무가설 기각
귀무가설 정규분포이다
MannwhitneyuResult(statistic=98.0, pvalue=0.14875840681321661)


기출 예제 27) 강의 상류와 하류의 생물 다양성 점수에 차이가 있는지 검정하시오. (단, 같은 강에서 상류와 하류는 서로 독립적이지 않으며, 종속적인 관계에 있다.)

In [32]:
from scipy.stats import ttest_rel
from scipy.stats import shapiro
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/27/river.csv')
display(df.head(4))


## 강의 상류와 하류는 독립이 아니기 떄문에, 대응표본 T검정

# 귀무 : 강의 상/하류에 따라 다양성 점수의 차이는 없다.
# 대립 : 강의 상/하류에 따라 다양성 점수의 차이가 있다.
stat, p_val = stats.ttest_rel(df.up, df.down)

print("귀무가설 기각" if p_val < 0.05 else "귀무가설 채택")

,River,up,down
0,A,57.1,45.7
1,B,51.6,37.2
2,C,53.9,53.3
3,D,59.0,54.3


귀무가설 채택


기출 예제 28) </br>
한 유저가 시식 전 물건 구매의사 유,무와 시식 후 구매의사 유,무에 대한 응답을 나타낸 데이터 {시식전} {구매의사 유 or 무} __ {시식후} {구매의사 유 or 무}

- 시식여부가 구매의사에 영향을 주는지 가설을 설정

In [34]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p5_.csv')
df.head()

,data,userId
0,시식전_유__시식후_유,user_1
1,시식전_유__시식후_유,user_2
2,시식전_유__시식후_유,user_3
3,시식전_유__시식후_유,user_4
4,시식전_유__시식후_유,user_5


In [46]:
# 귀무 : 시삭 유무에 따른 구매의사의 차이는 없다.
# 대립 : 시식 유무에 따른 구매의사는 차이가 있다.

df['before'] = df['data'].str.split('_').str[1].map({'유': '구매', '무': '비구매'})
df['after'] = df['data'].str.split('_').str[4].map({'유': '구매', '무': '비구매'})

from statsmodels.stats.contingency_tables  import mcnemar

tab = pd.crosstab(df['before'],df['after'])

mc = mcnemar(tab, correction= False)
print(mc.pvalue)

0.10775214433670044


기출 예제 28)
A,B 지역 학생의 점수에 차이가 있는지 가설을 설정

In [47]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/28/p6.csv')
df.head()

,score,school_name,ID
0,91.0,A,1
1,NaN,A,2
2,NaN,A,3
3,NaN,A,4
4,NaN,A,5


In [56]:
df2 = df.dropna()
display(df2.school_name.value_counts())

# 결측치 제거 후의 유효 데이터
# 비모수 2개 , 대응관계 아님 : 만 휘트니

# 귀무 : A,B지역 간 학생의 score 점수의 차이는 없다.
stat, p_val = mannwhitneyu(df2.query('school_name == "A"')['score'], df2.query('school_name == "B"')['score'])

print(p_val)

,count
school_name,
B,18
A,10


0.13048990112711467


기출 예제 31)
데이터 설명 : A,B,C,D,E 영업사원의 각 계약 성사 유무 (1:계약 , 0:미계약) 를 나타낸 데이터이다. 영업사원의 평균 계약 성사 건수는 같은지 통계 검정하라

In [61]:
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/31/adp_31_7.csv')
df = df.set_index('Unnamed: 0')
df.head(4)

,A,B,C,D,E
Unnamed: 0,,,,,
contract_1,1,0,1,1,1
contract_2,0,1,0,1,0
contract_3,1,0,0,0,0
contract_4,0,1,1,1,1


In [63]:
# H0 : 영업사원별 계약 성사 건수의 차이는 없다.
# H1 : 적어도 하나의 영업사원의 건수는 차이가 있다.

from statsmodels.stats.contingency_tables import cochrans_q
cochrans_q(df).pvalue

0.3406392758219471

연습 문제)
자동차 A,B,C에 대해 유저 9명의 선호도(1~6, 6은 가장선호,1은 가장 비선호)를 표시했다. 자동차 간의 선호도 차이가 존재하는지 검정하라

In [65]:
import pandas as pd

# 데이터 정의
data = {
    '차량 A': [4, 3, 2, 4, 4, 6, 5, 6, 2],
    '차량 B': [1, 4, 4, 3, 1, 4, 3, 3, 4],
    '차량 C': [6, 6, 6, 2, 6, 2, 5, 2, 6]
}
# 인덱스 정의
index = ['피험자1', '피험자2', '피험자3', '피험자4', '피험자5', '피험자6', '피험자7', '피험자8', '피험자9']

# DataFrame 생성
df = pd.DataFrame(data, index=index)

# 출력
print(df)

      차량 A  차량 B  차량 C
피험자1     4     1     6
피험자2     3     4     6
피험자3     2     4     6
피험자4     4     3     2
피험자5     4     1     6
피험자6     6     4     2
피험자7     5     3     5
피험자8     6     3     2
피험자9     2     4     6


In [72]:
## 3개의 대응관계 - 프리드만
from scipy.stats import friedmanchisquare
df2 = df.reset_index(drop = 'index')
stat , p = friedmanchisquare(df2.values[0] , df2.values[1], df2.values[2])

장르별 팝콘 구매여부에 관한 데이터이다. 장르별 팝콘 구매 여부가 연관이 있는지 확인하고 싶다.
1. 가설설정, 가설 검증
2. 장르과 구매여부의 정량적인 상관정도를 구하고 분석하라

In [73]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/mock/001_s2.csv')
df.head()

,유저ID,장르,구매여부
0,0,액션,구매
1,1,액션,구매
2,2,액션,구매
3,3,액션,구매
4,4,액션,구매


In [76]:
# 귀무 : 장르와 구매 여부는 독립이다.
# 대립 : 장르와 구매 여부는 독립이 아니다.

cross = pd.crosstab(df['장르'], df['구매여부'])

stat, p_val, dof, expect = chi2_contingency(cross)
print(p_val)

## 장르/ 구매여부의 정량적인 상관 관계 분석 - 크레마 계수

# Crema 계산 함수
n = len(df)
def cramer_v (chi2, n, k1, k2) :
  return np.sqrt(chi2 / (n * min(k1-1, k2-1)))

v = cramer_v(stat, n, cross.shape[0], cross.shape[1])
print(v)

4.987377462630171e-14
0.32916973606033734


예제 문제) 일자별 장마감시간의 주식의 상승과 하락을 추적한 데이터이다.
주가 변동이 무작위 인지 특정 경향성을 가지는지 통계적으로 확인하려 한다.

1. 가설설정
2. 검정 통계량 확인 및 분석

In [79]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/mock/003_s2.csv',index_col=0)
df.head()

,Price Change
2024-01-01,상승
2024-01-02,하락
2024-01-03,상승
2024-01-04,상승
2024-01-05,하락


In [90]:
sample = list(df.reset_index(drop = True)['Price Change'])

coded = [1 if x == '상승' else 0 for x in sample]

# 귀무가설 : 주식의 상승과 하락은 무작위이다.
# 대립가설 : 주식의 상승과 하락은 무작위가 아니다.

count, p_val = runstest_1samp(coded)
print(f"Number of runs" : {count})
print(f"Number of ru" : {p_val})

[1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1]

예제 문제)
연구자가 새로운 수면 개선 약을 개발했으며, 이 약의 효과를 검증하기 위해 10명의 피험자에게 일주일 동안 매일 밤 약을 복용하도록 하고, 복용 전과 복용 후의 평균 수면 시간을 비교했다. 연구자는 약이 수면 시간에 유의미한 변화를 가져다주었는지 확인하고 싶어 한다.      
(1) 가설 설정     
(2) 통계 검정

| 피험자 | 복용 전 평균 수면 시간(시간) | 복용 후 평균 수면 시간(시간) |
|---|------------------|------------------|
| 1 | 6 | 7 |
| 2 | 7 | 8 |
| 3 | 6.5 | 6.8 |
| 4 | 5 | 6.5 |
| 5 | 7 | 7.5 |
| 6 | 5.5 | 5 |
| 7 | 6.7 | 7 |
| 8 | 6.2 | 6.8 |
| 9 | 7 | 7.2 |
| 10 | 5.5 | 6 |


In [97]:
# 귀무 : 복용전후의 수면 시간의 차이는 없다.

# 대응 2개 , 비모수 : wilocox
data = {
    'before': [6, 7, 6.5, 5, 7, 5.5, 6.7, 6.2, 7, 5.5],
    'after': [7, 8, 6.8, 6.5, 7.5, 5, 7, 6.8, 7.2, 6]
}
df = pd.DataFrame(data)
print(df)

# 순위부호검정 (wilcoxon signed_rank test)
from scipy.stats import wilcoxon
stat, p = wilcoxon(df['before'], df['after'])
print(p)

   before  after
0     6.0    7.0
1     7.0    8.0
2     6.5    6.8
3     5.0    6.5
4     7.0    7.5
5     5.5    5.0
6     6.7    7.0
7     6.2    6.8
8     7.0    7.2
9     5.5    6.0
0.01953125


학생 20명을 무작위로 두 그룹으로 나누어 카페인이 학습에 효과가 있는지 확인하려한다. 두 그룹간의 평균 점수 차이가 존재하는지 확인하라    
카페인 그룹 테스트 점수: [88, 92, 94, 95, 96, 97, 91, 93, 98, 94]     
플라시보 그룹 테스트 점수: [81, 83, 84, 88, 86, 85, 87, 82, 80, 83]    

(1) 가설설정    
(2) 검정

모수 검정 (독립표본 t-검정)일 때:
- 귀무 가설 (H0): 카페인 그룹과 플라시보 그룹 간의 평균 점수 차이는 없다.
- 대립 가설 (H1): 카페인 그룹과 플라시보 그룹 간의 평균 점수 차이가 있다.

비모수 검정 (Mann-Whitney U 검정)일 때:
- 귀무 가설 (H0): 카페인 그룹과 플라시보 그룹 간의 점수 분포가 같다.
- 대립 가설 (H1): 카페인 그룹과 플라시보 그룹 간의 점수 분포가 다르다.

In [106]:
c =  [88, 92, 94, 95, 96, 97, 91, 93, 98, 94]
p =  [81, 83, 84, 88, 86, 85, 87, 82, 80, 83]

# 귀무 : 두 그룹간 평균 점수의 차이는 존재하지 않는다.

# 정규성 검토
# 귀무 : 정규 분포이다.
from scipy.stats import shapiro
shapiro(c)
shapiro(p)

print("카페인에 대한 정규분포이다 라는 귀무가설 기각" if shapiro(c).pvalue < 0.05 else " 카페인에 대한 정규분포이다 라는귀무가설 채택")
print("플라시보에 대한 정규분포이다 라는 귀무가설 기각" if shapiro(p).pvalue < 0.05 else "플라시보에 대한 정규분포이다 라는 귀무가설 채택")

from scipy.stats import ttest_ind
## 정규분포이다. - 모수 방법 선택
stat, p = ttest_ind(c, p)
print("두 그룹간 평균 점수의 차이는 존재하지 않는다라는 귀무가설 기각" if shapiro(c).pvalue < 0.05 else "두 그룹간 평균 점수의 차이는 존재하지 않는다라는  귀무가설 채택")

 카페인에 대한 정규분포이다 라는귀무가설 채택
플라시보에 대한 정규분포이다 라는 귀무가설 채택
두 그룹간 평균 점수의 차이는 존재하지 않는다라는  귀무가설 채택
